# Projeto 2 - NLP

## Grupo:
- Laianna Lana Virginio da Silva - lvs2@cin.ufpe.br
- Lucas Natan Correia Couri - lncc2@cin.ufpe.br
- Mariama Celi Serafim de Oliveira - mcso@cin.ufpe.br

# Importando bibliotecas

In [1]:
import preprocessing as pp
import load_files as lf
import nlp_algorithms as nlp

import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer

# Carregando conjunto de dados

In [2]:
pd.set_option('display.max_colwidth', -1)
## dfSourceV2 = pd.read_csv('../Dados/v2_semeval-2022_task8_train-data_batch.csv')
train_v1 = pd.read_csv('dados/train v0.1.csv')

train_v1_enen = train_v1[(train_v1['url1_lang'] == 'en') & (train_v1['url2_lang'] == 'en')]

train_v1_enen.head()

C:\Users\mcso\AppData\Local\Temp/ipykernel_16196/2099918564.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-man-arrested-in-fatal-dui-crash-in-west-virginia/2020/01/01/740fbc7a-2cbe-11ea-bffe-020c88b3f120_story.html,https://www.washingtonpost.com/world/the_americas/haitis-leader-marks-independence-day-amid-security-concerns/2020/01/01/dc4033a4-2cc5-11ea-bffe-020c88b3f120_story.html,https://web.archive.org/web/www.washingtonpost.com/local/virginia-man-arrested-in-fatal-dui-crash-in-west-virginia/2020/01/01/740fbc7a-2cbe-11ea-bffe-020c88b3f120_story.html,https://web.archive.org/web/www.washingtonpost.com/world/the_americas/haitis-leader-marks-independence-day-amid-security-concerns/2020/01/01/dc4033a4-2cc5-11ea-bffe-020c88b3f120_story.html,4.0,4.000000,1.000000,4.000000,4.000000,1.666667,2.000000
1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three-injured-after-car-crashes-into-utility-pole/,https://www.thestar.com/news/world/europe/2020/01/01/fire-kills-animals-at-zoo-in-western-germany.html,https://web.archive.org/web/www.stlucianewsonline.com/guyana-three-injured-after-car-crashes-into-utility-pole/,https://web.archive.org/web/www.thestar.com/news/world/europe/2020/01/01/fire-kills-animals-at-zoo-in-western-germany.html,4.0,4.000000,1.000000,4.000000,3.666667,1.666667,1.333333
2,en,en,1484698254_1483758694,https://www.teaparty.org/trump-brings-in-2020-at-mar-a-lago-were-going-to-have-a-great-year-423052/,https://www.timesofisrael.com/trump-says-he-does-not-expect-war-with-iran-likes-peace/,https://web.archive.org/web/www.teaparty.org/trump-brings-in-2020-at-mar-a-lago-were-going-to-have-a-great-year-423052/,https://web.archive.org/web/www.timesofisrael.com/trump-says-he-does-not-expect-war-with-iran-likes-peace/,1.0,2.000000,1.000000,2.333333,2.333333,1.000000,1.333333
3,en,en,1576314516_1576455088,https://gadgets.ndtv.com/apps/news/zomato-uber-eats-business-acquisition-india-all-stock-deal-2167155,https://gadgets.ndtv.com/internet/news/indian-online-food-delivery-market-to-hit-usd-8-billion-by-2020-google-bcg-report-2171043,https://web.archive.org/web/gadgets.ndtv.com/apps/news/zomato-uber-eats-business-acquisition-india-all-stock-deal-2167155,https://web.archive.org/web/gadgets.ndtv.com/internet/news/indian-online-food-delivery-market-to-hit-usd-8-billion-by-2020-google-bcg-report-2171043,1.0,2.333333,2.666667,1.666667,2.000000,1.666667,1.666667
4,en,en,1484036253_1483894099,https://news.yahoo.com/india-approves-third-moon-mission-085759387.html,https://www.channelnewsasia.com/news/asia/india-targets-new-moon-mission-in-2020-12225344,https://web.archive.org/web/news.yahoo.com/india-approves-third-moon-mission-085759387.html,https://web.archive.org/web/www.channelnewsasia.com/news/asia/india-targets-new-moon-mission-in-2020-12225344,1.0,1.250000,1.000000,1.250000,1.250000,1.000000,1.000000


In [3]:
data_path = 'dados/train v0.1/'

lista_docs = []
lista_error = []
lista_vazio = []
values = train_v1_enen[['pair_id', 'Overall']]

for index, values in values.iterrows():
    
    try:
        
        json_pair = lf.get_json_document_pair(data_path, values['pair_id'])
        text_doc1 = json_pair[0]['text']
        text_doc2 = json_pair[1]['text']
        
        if ( len(text_doc1) > 0 ) and ( len(text_doc2) > 0 ):
            lista_docs.append((values['pair_id'], text_doc1, text_doc2, values['Overall']))
        else:
            lista_vazio.append(values['pair_id'])
    
    except:
        lista_error.append(values['pair_id'])

In [4]:
df_text = pd.DataFrame(lista_docs,  columns=['pair_id', 'doc1', 'doc2', 'Overall'])
#df_text.head(1)

In [5]:
df_text.shape

(1357, 4)

# Pré-processamento dos dados

In [6]:
pre_processing_list = [
    {},
    {"basic_processing": True},
    {"basic_processing": True, "no_stopwords": True},
    {"basic_processing": True, "stemming": True},
    {"basic_processing": True, "no_stopwords": True, "stemming": True},
    {"basic_processing": True, "lema":  True},
    {"basic_processing": True, "no_stopwords": True, "lema": True}]

#df_pp = pp.pre_process_all(df_text, pre_processing_list)
df_pp = pp.pre_process_all(df_text.iloc[:10], pre_processing_list)

c:\Users\mcso\Desktop\residencia\NLP\projeto2-nlp-sidi-residencia\preprocessing.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"doc1_pipeline{index}"] = df["doc1"].apply(lambda x: pre_process(x, **param))
c:\Users\mcso\Desktop\residencia\NLP\projeto2-nlp-sidi-residencia\preprocessing.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"doc2_pipeline{index}"] = df["doc2"].apply(lambda x: pre_process(x, **param))


In [7]:
#df_pp.head(1)

# Algoritmos

In [8]:
df_results = df_pp[["pair_id", "Overall"]]
df_results.head(1)

,pair_id,Overall
0,1484084337_1484110209,4.0


## Bag of Words (CountVectorizer)

In [9]:
df_bow = nlp.apply_bow(df_pp, len(pre_processing_list))

In [10]:
df_results = df_results.join(df_bow)

In [11]:
df_results.head(5)

,pair_id,Overall,bow0,bow1,bow2,bow3,bow4,bow5,bow6
0,1484084337_1484110209,4.000000,0.554412,0.568225,0.118185,0.567762,0.118088,0.568225,0.117200
1,1484396422_1483924666,3.666667,0.658932,0.662808,0.077949,0.652465,0.078233,0.658427,0.078285
2,1484698254_1483758694,2.333333,0.642877,0.652648,0.395561,0.658897,0.410352,0.689574,0.465636
3,1576314516_1576455088,2.000000,0.678074,0.683262,0.461996,0.683147,0.482403,0.684565,0.475937
4,1484036253_1483894099,1.250000,0.790918,0.780586,0.479421,0.799176,0.540089,0.799223,0.535830


In [12]:
df_bow.to_csv('./resultados/bow.csv', index = False)

## TF-IDF

In [ ]:
df_tf_idf = nlp.apply_tf_idf(df_pp, len(pre_processing_list))

In [ ]:
df_results = df_results.join(df_tf_idf)

In [ ]:
df_results.head(5)

In [ ]:
df_tf_idf.to_csv('./resultados/tf_idf.csv', index = False)

## BERT

In [ ]:
'''model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')
model_name = 'base_mean'

df_bert = nlp.apply_bert(df_pp, len(pre_processing_list), model, model_name)

df_results = df_results.join(df_bert)'''

In [ ]:
df_bert.to_csv('./resultados/bert_mean.csv', index = False)

In [ ]:
'''model = SentenceTransformer('sentence-transformers/bert-base-nli-max-tokens')
model_name = 'base_max'

df_bert = nlp.apply_bert(df_pp, len(pre_processing_list), model, model_name)

df_results = df_results.join(df_bert)'''

In [ ]:
df_bert.to_csv('./resultados/bert_.csv', index = False)

In [ ]:
'''model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
model_name = 'miniLM'

df_bert = nlp.apply_bert(df_pp, len(pre_processing_list), model, model_name)

df_results = df_results.join(df_bert)'''

In [ ]:
#df_results.head(10)

In [ ]:
df_bert.to_csv('./resultados/bert.csv', index = False)

## Doc2vec

In [ ]:
#Treinamento dos modelos
#nlp.train_doc2vec(df_pp, len(pre_processing_list))

In [ ]:
df_doc2vec = nlp.apply_doc2vec(df_pp, len(pre_processing_list))

In [ ]:
df_results = df_results.join(df_doc2vec)

In [ ]:
df_results.head(10)

In [ ]:
df_doc2vec.to_csv('./resultados/doc2vec.csv', index = False)

## Word2vec

In [ ]:
import gensim.downloader as api

In [ ]:
model = api.load('word2vec-google-news-300')

In [ ]:
df_word2vec = nlp.apply_word2vec(df_pp, len(pre_processing_list), model)

In [ ]:
df_results = df_results.join(df_word2vec)

In [ ]:
df_results.head(10)

In [ ]:
df_word2vec.to_csv('./resultados/word2vec.csv', index = False)